In [ ]:
!pip install requests
!pip install bs4
!pip install lxml
!pip install pandas

In [1]:
# import libraries
import os
import pandas as pd
import lxml
import requests  
from bs4 import BeautifulSoup, Comment

In [14]:
def read_txt_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            return lines
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

match_url_list = read_txt_file(os.getcwd() + '\data\Ascension.txt')

In [2]:
# In order to get the list of match of the all VLR Season
match_url_list = []

source_matchlist = requests.get(url='https://www.vlr.gg/event/matches/449/valorant-champions/?series_id=all').text

soup_matchlist = BeautifulSoup(source_matchlist,'lxml')

days = soup_matchlist.findAll('div', {'class':'wf-card'})

for d in range(1,len(days)):
    for i in days[d].findAll(href=True):
        match_url_list.append(i['href'])

In [17]:
from utils import *
match_url_list = ['https://www.vlr.gg/237652/apeks-vs-gentle-mates-champions-tour-2023-emea-ascension-gf/?game=all&tab=economy']
match_stats = []
row1, row2 = [], []

for matchnum in range(len(match_url_list)):
    url = match_url_list[matchnum]
    
    source_match = requests.get(url=url).text
    soup_match = BeautifulSoup(source_match, features="html.parser")

    stage = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split(":", 1)[0]

    series = soup_match.findAll('div', {'class':'match-header-event-series'})[0].text.strip().split("\n", 1)[1].strip()

    # for table_economy[i] if i%2==0 : the full bank with the round description, otherwise that is the higher level information
    # the last one is not useful as it is the overall result so only things interesting are [0,5] for a BO3
    table_economy = soup_match.findAll('table', {'class':'wf-table-inset mod-econ'})

    for i in range(0, len(table_economy)-1, 2):

        map_num = i // 2
        map_name = soup_match.findAll('div', {'class':'vm-stats-gamesnav-item js-map-switch'})[map_num].text.strip()[1:].strip()

        headers_match = ["Team Name", "Map #", "Map Name", "Stage", "Series", "Pistol_Won", "Eco", "Eco_Won", "$", "$_Won", "$$", "$$_Won", '$$$', '$$$_Won', "Bank", "Buys"]
        df_match = pd.DataFrame(columns=headers_match)

        table_economy_general = table_economy[i]
        bank = table_economy[i+1]

        team1, team2 = get_economy_data(table_economy_general)
        banks, buys = get_banking_data(bank)

        length = len(df_match)

        df_match.loc[length], df_match.loc[length+1] = create_economy_row(team1, team2, banks, buys, series, stage, map_num, map_name)

        match_stats.append(df_match)

result = pd.concat(match_stats).reset_index(drop=True)
result = result.apply(pd.to_numeric, errors='ignore')
        


C:\Users\marius.reymauzaize\AppData\Local\Temp\ipykernel_5404\3337209969.py:43: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  comment_text = [comment.string.strip() for comment in column.find_all(text=lambda text: isinstance(text, Comment))]
C:\Users\marius.reymauzaize\AppData\Local\Temp\ipykernel_5404\3337209969.py:43: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  comment_text = [comment.string.strip() for comment in column.find_all(text=lambda text: isinstance(text, Comment))]
C:\Users\marius.reymauzaize\AppData\Local\Temp\ipykernel_5404\3337209969.py:43: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  comment_text = [comment.string.strip() for comment in column.find_all(text=lambda text: isinstance(text, Comment))]


# scraping of the performance info